# Projecto Integrador

Primer acercamiento a trabajar con el concepto de web Scrapping. El proyecto fué propuesto en clase por el maestro Gonzalo del Rio, durante el primer módulo del Bootcamp de Henry.

El proyecto consiste en hacer WebScrapping al sitio web https://cuspide.com/ para extraer la información de la sección de "100 libros más vendidos de la semana" en un formato permita hacer el procesamiento con Pandas y crear finalmente un archivo csv de salida.

***

### Para empezar:

#### 1) Preparar el ambiente. 

Se importan las librerías de procesamiento de los datos con las que se va a trabajar.

- Se usa Pandas para el procesamiento y visualización de los datos. **Se requiere crear un DataFrame.**
- Se usa el módulo `requests` para enviar requerimentos HTTP al sitio que nos interesa. Como respuesta queremos obtener el HTML crudo de la página.
- Se usa `Beautiful Soup` para leer el HTML en una estructura que sea útil para extraer la información.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# Define a variable containing the website's url.
url = 'https://cuspide.com/100-mas-vendidos/' 

# Making the HTTP request using request.get() method.
# A response object is returned.
response = requests.get(url) 

# Getting raw html from response.content attribute.
html_content = response.content

### 2) BeautifulSoup 

Beautiful soup interpreta el contenido html crudo y lo organiza en estructura jerárquica, parecido a como lo hace el navegador. Con la estructura de etiquetas donde podemos analizar esa jerarquía ya se puede empezar a estraer la información. 

Esa es la magia de Beautiful soup :D

In [4]:
# Parsing the content with bs4
soup = BeautifulSoup(html_content, 'html.parser')

Luego de inspeccionar la página, se observó que los titulos están contenidos en una etiqueta `<h3>` con el atributo de clase "product-title".

**Nota**: El método `find_all()` retorna un objeto tipo ResultSet que contiene todas las etiquetas que cumplan con los parámetros que se pasen por la función. Este ResultSet se puede iterar para trabajar con cada etiqueta específica.

In [5]:
# Finding title's tags for each book
title_tags = soup.find_all("h3", class_='product-title')

for tag in title_tags: # Loop over each tag
   print(tag.get_text(strip=True)) # Printing text within every tag

LA CASA NEVILLE . LA FORMIDABLE SEÑORITA  MANON
LA MUJER QUE SOY
ESTE DOLOR NO ES MIO
OXIDO
EL PROBLEMA FINAL
HABITOS ATOMICOS
ARTIFICIAL
HOROSCOPO CHINO 2024
EL PODER DE LAS PALABRAS
EL VIENTO CONOCE MI NOMBRE
HOLLY
TESIS SOBRE UNA DOMESTICACION
UNA FAMILIA ANORMAL
DESTROZA ESTE DIARIO ( A TODO COLOR )
MANNY MANITAS  RECORTA Y PEGA CON PEGATINAS
TAYLOR : FROM THE VAULT
LOS CUATRO ACUERDOS
LAS NI/AS DEL NARANJEL
DIBU MARTINEZ : PASION POR EL FUTBOL
LA PACIENTE SILENCIOSA
EL HOMBRE EN BUSCA DE SENTIDO
DEJA DE SER TU
LOS OJOS PLATA  FIVE NIGHTS AT FREDDY'S
LA FELICIDAD CABE EN UNA TAZA DE CAFE
LA ARMADURA DE LA LUZ
MI DIARIO SECRETO : ROSA
BEYOND THE STORY ( EDICION EN ESPA/OL )
DESARMA ESTE LIBRO
EL MONJE QUE VENDIO SU FERRARI
EL VUELO DE LA LIBELULA
MIS DIAS EN LA LIBRERIA MORISAKI
CRONICAS MUNDIALES LAS MEJORES ANECDOTAS DE LA COPA DEL MUNDO (2023)
UN VECINO ANORMAL  : Y EL LADRON DEL CHOCOLATE
NOSOTROS DOS EN LA TORMENTA
LA CASA DE LOS SUICIDIOS
PADRE RICO , PADRE POBRE ( 25 AÑOS )
E

Teniendo las etiquetas, podemos guardarlas en una lista (usando list comprehension).

In [6]:
# Making a list with titles
titulos = [tag.get_text(strip=True).title() for tag in title_tags]
titulos

['La Casa Neville . La Formidable Señorita  Manon',
 'La Mujer Que Soy',
 'Este Dolor No Es Mio',
 'Oxido',
 'El Problema Final',
 'Habitos Atomicos',
 'Artificial',
 'Horoscopo Chino 2024',
 'El Poder De Las Palabras',
 'El Viento Conoce Mi Nombre',
 'Holly',
 'Tesis Sobre Una Domesticacion',
 'Una Familia Anormal',
 'Destroza Este Diario ( A Todo Color )',
 'Manny Manitas  Recorta Y Pega Con Pegatinas',
 'Taylor : From The Vault',
 'Los Cuatro Acuerdos',
 'Las Ni/As Del Naranjel',
 'Dibu Martinez : Pasion Por El Futbol',
 'La Paciente Silenciosa',
 'El Hombre En Busca De Sentido',
 'Deja De Ser Tu',
 "Los Ojos Plata  Five Nights At Freddy'S",
 'La Felicidad Cabe En Una Taza De Cafe',
 'La Armadura De La Luz',
 'Mi Diario Secreto : Rosa',
 'Beyond The Story ( Edicion En Espa/Ol )',
 'Desarma Este Libro',
 'El Monje Que Vendio Su Ferrari',
 'El Vuelo De La Libelula',
 'Mis Dias En La Libreria Morisaki',
 'Cronicas Mundiales Las Mejores Anecdotas De La Copa Del Mundo (2023)',
 'Un Vecin


... Por inspección tambíen puede verse que los url de cada libro están contenidos en una etiqueta "hija" dentro de las mismas etiquetas de los títulos.

**Nota**: El objeto etiqueta nos permite acceder a las etiquetas que tiene por dentro (estructura jerárquica de arbol). Con el atritubo .attrs podemos ver un diccionario con los atributos de la etiqueta.

In [7]:
# Making a list with urls for every book detail
urls = [tag.a.attrs['href'] for tag in title_tags]
urls

['https://cuspide.com/producto/la-casa-neville-la-formidable-senorita-manon/',
 'https://cuspide.com/producto/la-mujer-que-soy/',
 'https://cuspide.com/producto/este-dolor-no-es-mio/',
 'https://cuspide.com/producto/oxido/',
 'https://cuspide.com/producto/el-problema-final/',
 'https://cuspide.com/producto/habitos-atomicos-2/',
 'https://cuspide.com/producto/artificial-2/',
 'https://cuspide.com/producto/horoscopo-chino-2024/',
 'https://cuspide.com/producto/el-poder-de-las-palabras/',
 'https://cuspide.com/producto/el-viento-conoce-mi-nombre/',
 'https://cuspide.com/producto/holly/',
 'https://cuspide.com/producto/tesis-sobre-una-domesticacion/',
 'https://cuspide.com/producto/una-familia-anormal-5/',
 'https://cuspide.com/producto/destroza-este-diario-a-todo-color/',
 'https://cuspide.com/producto/manny-manitas-recorta-y-pega-con-pegatinas/',
 'https://cuspide.com/producto/taylor-from-the-vault/',
 'https://cuspide.com/producto/los-cuatro-acuerdos/',
 'https://cuspide.com/producto/la

#### Para los precios se usó un método de distinto (para ilustrar que hay diferentes formas de hacerlo).

En lugar de find_all, puede usarse el método selector de css que de igual manera es muy cómodo para seleccionar las etiquetas por clases, estilos y id.

In [8]:
# Selecting tags by css classes
price_tags = soup.css.select('.product-price')

for tag in price_tags:
    print(tag.get_text(strip=True))

$13.700,00
$8.999,00
$11.500,00
$14.999,00
$13.999,00
$16.000,00
$9.999,00
$7.999,00
$13.899,00
$13.999,00
$18.999,00
$11.800,00
$5.799,00
$12.900,00
$400,00
$8.299,00
$10.690,00
$10.999,00
$6.699,00
$12.999,00
$7.590,00
$16.890,00
$12.799,00
$12.799,00
$24.999,00
$8.599,00
$14.999,00
$4.399,00
$13.899,00
$12.999,00
$7.390,00
$4.999,00
$6.399,00
$15.999,00
$11.900,00
$14.899,00
$14.000,00
$9.499,00
$9.499,00
$800,00
$7.490,00
$8.999,00
$400,00
$400,00
$18.600,00
$12.799,00
$6.490,00
$9.990,00
$11.900,00
$580,00
$580,00
$13.550,00
$8.150,00
$16.000,00
$3.999,00
$8.399,00
$580,00
$7.599,00
$9.600,00
$10.400,00
$12.900,00
$13.899,00
$400,00
$580,00
$9.860,00
$2.599,00
$400,00
$600,00
$7.790,00
$12.599,00
$9.150,00
$8.399,00
$13.000,00
$4.750,00
$9.100,00
$7.990,00
$10.999,00
$12.000,00
$4.990,00
$11.800,00
$6.990,00
$6.830,00
$4.699,00
$10.290,00
$15.200,00
$8.900,00
$400,00
$17.599,00
$10.400,00
$8.700,00
$9.500,00
$12.999,00


In [9]:
# Making list for prices. Getting only numeric part.
price_column = [tag.get_text(strip=True)[1:] for tag in price_tags]

# Formatting and casting to float.
price_column = [float(price.replace(".","").replace(",",".")) for price in price_column]
price_column

[13700.0,
 8999.0,
 11500.0,
 14999.0,
 13999.0,
 16000.0,
 9999.0,
 7999.0,
 13899.0,
 13999.0,
 18999.0,
 11800.0,
 5799.0,
 12900.0,
 400.0,
 8299.0,
 10690.0,
 10999.0,
 6699.0,
 12999.0,
 7590.0,
 16890.0,
 12799.0,
 12799.0,
 24999.0,
 8599.0,
 14999.0,
 4399.0,
 13899.0,
 12999.0,
 7390.0,
 4999.0,
 6399.0,
 15999.0,
 11900.0,
 14899.0,
 14000.0,
 9499.0,
 9499.0,
 800.0,
 7490.0,
 8999.0,
 400.0,
 400.0,
 18600.0,
 12799.0,
 6490.0,
 9990.0,
 11900.0,
 580.0,
 580.0,
 13550.0,
 8150.0,
 16000.0,
 3999.0,
 8399.0,
 580.0,
 7599.0,
 9600.0,
 10400.0,
 12900.0,
 13899.0,
 400.0,
 580.0,
 9860.0,
 2599.0,
 400.0,
 600.0,
 7790.0,
 12599.0,
 9150.0,
 8399.0,
 13000.0,
 4750.0,
 9100.0,
 7990.0,
 10999.0,
 12000.0,
 4990.0,
 11800.0,
 6990.0,
 6830.0,
 4699.0,
 10290.0,
 15200.0,
 8900.0,
 400.0,
 17599.0,
 10400.0,
 8700.0,
 9500.0,
 12999.0]

Debido a que el precio en dólares se encuentra en cada url específico para cada libro, he creado funciones scrapper para esta tarea.
Las funciones están definidas dentro del módulo `functions.py`. Allí puede verse el script que se ha creado.

In [ ]:
from functions import get_usdprice_and_date

# Getting usd price for each book.
usd_prices = []
dates = [] 
for url in urls:
    usd_price, date = get_usdprice_and_date(url)
    usd_prices.append(usd_price)
    dates.append(date)

['29/09/2023',
 '31/10/2023',
 '16/03/2018',
 '06/10/2023',
 '28/09/2023',
 '27/05/2019',
 '28/09/2023',
 '28/09/2023',
 '29/08/2022',
 '06/06/2023',
 '28/09/2023',
 '31/08/2023',
 '28/07/2023',
 '02/03/2018',
 '03/05/2016',
 '28/09/2023',
 '16/10/2007',
 '28/09/2023',
 '29/09/2022',
 '25/01/2022',
 '21/04/2016',
 '14/01/2021',
 '29/05/2017',
 '29/03/2023',
 '28/09/2023',
 '20/05/2023',
 '29/08/2023',
 '07/09/2023',
 '11/04/2022',
 '29/08/2023',
 '31/01/2023',
 '19/10/2023',
 '28/09/2023',
 '27/05/2023',
 '31/08/2023',
 '12/10/2023',
 '04/10/2021',
 '22/01/2021',
 '31/08/2015',
 '03/05/2016',
 '06/06/2023',
 '28/09/2023',
 '03/05/2016',
 '03/05/2016',
 '06/12/2019',
 '28/11/2022',
 '09/02/2023',
 '28/09/2023',
 '04/10/2023',
 '26/11/2015',
 '26/11/2015',
 '15/12/2020',
 '26/07/2022',
 '03/08/2022',
 '21/12/2021',
 '29/03/2023',
 '26/11/2015',
 '27/04/2023',
 '31/07/2023',
 '03/03/2023',
 '05/05/2023',
 '18/10/2019',
 '03/05/2016',
 '26/11/2015',
 '23/02/2023',
 '19/10/2023',
 '03/05/20

In [ ]:
# Formatting and casting usd prices
usd_prices = [float(price.replace(",",".")) for price in usd_prices]
usd_prices[:5]

AttributeError: 'list' object has no attribute 'replace'

Obteniendo el precio del Dolar Blue:

In [ ]:
r = requests.get('https://www.cronista.com/MercadosOnline/dolar.html')
dom_content = r.text
html_soup = BeautifulSoup(dom_content, 'html.parser')

dolar_blue = html_soup.find('span', string='Dólar Blue')

In [ ]:
# Span tag containing value is the next one. "The sibling"
dolar_blue.next_sibling.text[1:]

'925,00'

In [ ]:
# Formatting and casting.
dolarBlue_precio = float(dolar_blue.next_sibling.text[1:].replace(",","."))
dolarBlue_precio

925.0

In [ ]:
# Making list with dolar blue conversion.
blue_column = [round(precio / dolarBlue_precio, 2) for precio in price_column]
blue_column[:5]

[14.81, 9.73, 12.43, 16.22, 15.13]

***
### Creación del DataFrame:

- The following step is to create a .csv output file.
- Using Pandas for this purpose.

In [ ]:
# Creating a pandas DataFrame
df = pd.DataFrame(
    {
        'titulo': [str(title).title() for title in titulos],
        'fecha_publicación': pd.to_datetime(dates, yearfirst=True),
        'url': [str(item) for item in urls],
        'precio_pesos': pd.Series(price_column),
        'precio_usd': pd.Series(usd_prices),
        'precio_dolar_blue': pd.Series(blue_column)
    }
)
df.head(10)

,titulo,url,precio_pesos,precio_usd,precio_dolar_blue,fecha_publicación
0,La Casa Neville . La Formidable Señorita Manon,https://cuspide.com/producto/la-casa-neville-l...,13700.0,37.48,14.81,2023-09-29
1,La Mujer Que Soy,https://cuspide.com/producto/la-mujer-que-soy/,8999.0,24.62,9.73,2023-10-31
2,Este Dolor No Es Mio,https://cuspide.com/producto/este-dolor-no-es-...,11500.0,31.46,12.43,2018-03-16
3,Oxido,https://cuspide.com/producto/oxido/,14999.0,41.04,16.22,2023-06-10
4,El Problema Final,https://cuspide.com/producto/el-problema-final/,13999.0,38.30,15.13,2023-09-28
5,Habitos Atomicos,https://cuspide.com/producto/habitos-atomicos-2/,16000.0,43.78,17.30,2019-05-27
6,Artificial,https://cuspide.com/producto/artificial-2/,9999.0,27.36,10.81,2023-09-28
7,Horoscopo Chino 2024,https://cuspide.com/producto/horoscopo-chino-2...,7999.0,21.89,8.65,2023-09-28
8,El Poder De Las Palabras,https://cuspide.com/producto/el-poder-de-las-p...,13899.0,38.03,15.03,2022-08-29
9,El Viento Conoce Mi Nombre,https://cuspide.com/producto/el-viento-conoce-...,13999.0,38.30,15.13,2023-06-06


In [ ]:
# Creating the output file
df.to_csv('./libros_semana.csv', index=False)

*** 
2da Parte del proyecto - Módulo 3

## Conexión con MySQL: Creando una base de datos.

En esta sección se crea una base de datos MySQL en lugar de un archivo csv. También se implementa manejo de excepciones para crear una tabla de errores cuando los haya.

In [ ]:
import pymysql

In [ ]:
# Creating connection
connection = pymysql.Connection(
    host='localhost',
    user='juancml',
    passwd='',
    database='libros'
)

# creating mysql cursor
cursor = connection.cursor()
# Query to insert values on mysql table.
query = """ INSEERT INTO libros_semana 
            VALUES (%s,%s,%s,%s,%s,%s, CURRDATE()) """

El dataFrame debe convertirse a un listado de tuplas donde cada una de ellas es una fila que contiene los datos.

In [ ]:
# Turning DataFrame into a list of tuples.
table = [tuple(df.iloc[row]) for row in range(len(df))]

Se hace la insercion de los datos con el método del cursor `executemany()`.

In [ ]:
cursor.executemany(query, table)
# Commit changes and close database connection
connection.commit()
connection.close()